In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy.special as sc

In [2]:
# Data generation
X=1/np.random.gamma(2,2,3200)
A = np.random.gamma(2,2,3200)
Y=A*X
Y_log =np.log(Y)

In [3]:
print(Y)
print(Y_log)

[2.19197583 0.41095823 1.56918962 ... 2.64840921 1.21761171 0.92721729]
[ 0.78480334 -0.88926369  0.45055932 ...  0.97395916  0.19689133
 -0.07556733]


# Version 1

# Gaussian VAE

In [4]:
features = 1
# define a simple linear VAE
class LinearVAE(nn.Module):
    def __init__(self):
        super(LinearVAE, self).__init__()
        
 
        # encoder
        self.enc1 = nn.Linear(in_features=1, out_features=5)
        self.enc2 = nn.Linear(in_features=5, out_features=features*2)
 
        # decoder 
        self.dec1 = nn.Linear(in_features=features, out_features=5)
        self.dec2 = nn.Linear(in_features=5, out_features=features*2)
    
    def reparameterize(self, mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5*log_var) # standard deviation
        eps = torch.randn_like(std) # `randn_like` as we need the same size
        sample = mu + (eps * std) # sampling as if coming from the input space
        return sample
 
    def forward(self, x):
        # encoding
        x = F.relu(self.enc1(x))
        x = self.enc2(x).view(-1, 2, features)
        # get `mu` and `log_var`
        mu = x[:, 0, :] # the first feature values as mean
        log_var = x[:, 1, :] # the other feature values as variance
        # get the latent vector through reparameterization
        z = self.reparameterize(mu, log_var)
 
        # decoding
        x = F.relu(self.dec1(z))
        reconstruction = self.dec2(x).view(-1, 2, features)
        mu_theta = reconstruction[:, 0, :]
        log_var_theta = reconstruction[:, 1, :] 
        return mu_theta,log_var_theta, mu, log_var

In [14]:
import torch
import torchvision
import torch.optim as optim
import argparse
import matplotlib
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
#import model
from tqdm import tqdm
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import math
import mpmath
matplotlib.style.use('ggplot')

In [5]:
# leanring parameters
epochs = 1000
batch_size = 64
lr = 0.0001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
#Prepare dataset for training
train_data = torch.utils.data.TensorDataset((torch.from_numpy(Y[:2880])).float())
print(train_data)
val_data = torch.utils.data.TensorDataset((torch.from_numpy(Y[2880:])).float())
# training and validation data loaders
train_loader = DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)
val_loader = DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=False
)

In [120]:
model1 = LinearVAE().to(device)
optimizer = optim.Adam(model1.parameters(), lr=lr)

def entropy_loss(data, mut,logvart):  
    cost = torch.sum((data-mut)**2/(2*torch.exp(logvart)**2)+logvart)
    return(cost)
'''    
def entropy_sum_loss(data,mu,logvar, mut,logvart):
    n_el =100
    epsilon = torch.randn(data.shape,100)
    samplex = mu+torch.exp(logvar)*epsilon
    cost = 1/n_el torch.sum()
    '''

'    \ndef entropy_sum_loss(data,mu,logvar, mut,logvart):\n    n_el =100\n    epsilon = torch.randn(data.shape,100)\n    samplex = mu+torch.exp(logvar)*epsilon\n    cost = 1/n_el torch.sum()\n    '

In [121]:
def final_loss(ES_loss, mu, logvar):
    """
    This function will add the reconstruction loss (BCELoss) and the 
    KL-Divergence.
    KL-Divergence = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    :param bce_loss: recontruction loss
    :param mu: the mean from the latent vector
    :param logvar: log variance from the latent vector
    """
    ESL = ES_loss 
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return ES_loss + KLD

In [122]:
def fit(model, dataloader):
    model.train()
    running_loss = 0.0
    for i, batch in tqdm(enumerate(dataloader), total=int(len(train_data)/dataloader.batch_size)):
        batch=batch[0]
        batch= batch
        batch = batch.to(device)
        batch = batch.view(batch.size(0), -1)
        optimizer.zero_grad()
        mut,logvart, mu, logvar = model(batch)
        ES_loss = entropy_loss(batch,mut,logvart)
        loss = final_loss(ES_loss, mu, logvar)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    train_loss = running_loss/len(dataloader.dataset)
    return train_loss

In [123]:
def validate(model, dataloader):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for i, batch in tqdm(enumerate(dataloader), total=int(len(val_data)/dataloader.batch_size)):            
            batch=batch[0]
            batch = batch.to(device)
            batch = batch.view(batch.size(0), -1)
            mut,logvart, mu, logvar = model(batch)
            ES_loss = entropy_loss(batch,mut,logvart)
            loss = final_loss(ES_loss, mu, logvar)
            running_loss += loss.item()
        
            # save the last batch input and output of every epoch
            #if i == int(len(val_data)/dataloader.batch_size) - 1:
             #   num_rows = 8
              #  both = torch.cat((data.view(batch_size, 1, 28, 28)[:8], 
               #                   reconstruction.view(batch_size, 1, 28, 28)[:8]))
              #  save_image(both.cpu(), f"../outputs/output{epoch}.png", nrow=num_rows)
    val_loss = running_loss/len(dataloader.dataset)
    return val_loss

In [124]:
train_loss = []
val_loss = []
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    print(train_loader)
    train_epoch_loss = fit(model1, train_loader)
    val_epoch_loss = validate(model1, val_loader)
    train_loss.append(train_epoch_loss)
    val_loss.append(val_epoch_loss)
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f"Val Loss: {val_epoch_loss:.4f}")

  0%|          | 0/45 [00:00<?, ?it/s]

Epoch 1 of 1000
Train Loss: 2.1176
Val Loss: 1.9623
Epoch 2 of 1000


  0%|          | 0/45 [00:00<?, ?it/s]

Train Loss: 2.0054
Val Loss: 1.9120
Epoch 3 of 1000
Train Loss: 1.9927
Val Loss: 1.8838
Epoch 4 of 1000


  0%|          | 0/45 [00:00<?, ?it/s]

...

  0%|          | 0/45 [00:00<?, ?it/s]

Train Loss: 1.3672
Val Loss: 1.3020
Epoch 35 of 1000
Train Loss: 1.3602
Val Loss: 1.2839
Epoch 36 of 1000


 58%|█████▊    | 26/45 [00:00<00:00, 276.25it/s]

Train Loss: 1.3628
Val Loss: 1.2944
Epoch 37 of 1000


KeyboardInterrupt: 

# Inv-Gamma VAE

We set alpha = 2 because according to our model, the shape parameter of the obs and hidden state is equal. We use a hyperparameters beta=0.5 .
For the sake of the simplicity and to use the decomposition of gamma law as sum of exponential we set alphaq =2

In [ ]:
features = 1
# define a simple linear VAE
class LinearVAE_invG(nn.Module):
    def __init__(self):
        super(LinearVAE_invG, self).__init__()
        
        self.beta_p = 0.5
 
        # encoder
        self.enc1 = nn.Linear(in_features=1, out_features=5)
        self.enc2 = nn.Linear(in_features=5, out_features=features)
 
        # decoder 
        self.dec1 = nn.Linear(in_features=features, out_features=5)
        self.dec2 = nn.Linear(in_features=5, out_features=features*2)
    
    def reparameterize(self, beta):
        """
        :param alpha: 2
        :param beta:  from the encoder's latent space
        """
        alpha = 2 
        u1 = torch.rand_like(beta)
        u2 =torch.rand_like(beta)
        
        sample = -1/(beta*(torch.log(1-u1)+torch.log(1-u2))) # sampling as if coming from the input space
        return sample
 
    def forward(self, x):
        # encoding
        x = F.relu(self.enc1(x))
        x = self.enc2(x)
        # get `mu` and `log_var`
        beta = x**2 
        z = self.reparameterize(beta)
        # decoding
        x = F.relu(self.dec1(z))
        reconstruction = self.dec2(x).view(-1, 2, features)
        alpha_theta = reconstruction[:, 0, :]**2
        beta_theta = reconstruction[:, 1, :] **2
        return beta, alpha_theta, beta_theta

In [ ]:
model2 = LinearVAE_invG().to(device)
optimizer = optim.Adam(model2.parameters(), lr=lr)

def entropy_loss(data, alpha_t,beta_t):  
    cost = torch.mean((alpha_t-1)* torch.log(data)-beta_t*data+alpha_t*torch.log(beta_t) -torch.lgamma(alpha_t))
    return(cost)

In [ ]:
def final_loss(ES_loss, beta_q):
    """
    This function will add the reconstruction loss (BCELoss) and the 
    KL-Divergence.
    KL-Divergence = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    :param bce_loss: recontruction loss
    :param mu: the mean from the latent vector
    :param logvar: log variance from the latent vector
    """
    ESL = -ES_loss 
    KLD = torch.mean(4*(torch.log(beta_q))+1/beta_q)
    return ES_loss + KLD

In [ ]:
def fit(model, dataloader):
    model.train()
    running_loss = 0.0
    for i, batch in tqdm(enumerate(dataloader), total=int(len(train_data)/dataloader.batch_size)):
        batch=batch[0]
        batch= batch
        batch = batch.to(device)
        batch = batch.view(batch.size(0), -1)
        optimizer.zero_grad()
        beta_q, alpha_t, beta_t = model(batch)
        ES_loss = entropy_loss(batch,alpha_t,beta_t)
        loss = final_loss(ES_loss, beta_q)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    train_loss = running_loss/len(dataloader.dataset)
    return train_loss

In [ ]:
def validate(model, dataloader):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for i, batch in tqdm(enumerate(dataloader), total=int(len(val_data)/dataloader.batch_size)):            
            batch=batch[0]
            batch = batch.to(device)
            batch = batch.view(batch.size(0), -1)
            beta_q, alpha_t, beta_t = model(batch)
            ES_loss = entropy_loss(batch,alpha_t,beta_t)
            loss = final_loss(ES_loss, beta_q)
            running_loss += loss.item()
        
            # save the last batch input and output of every epoch
            #if i == int(len(val_data)/dataloader.batch_size) - 1:
             #   num_rows = 8
              #  both = torch.cat((data.view(batch_size, 1, 28, 28)[:8], 
               #                   reconstruction.view(batch_size, 1, 28, 28)[:8]))
              #  save_image(both.cpu(), f"../outputs/output{epoch}.png", nrow=num_rows)
    val_loss = running_loss/len(dataloader.dataset)
    return val_loss

In [ ]:
train_loss = []
val_loss = []
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    print(train_loader)
    train_epoch_loss = fit(model2, train_loader)
    val_epoch_loss = validate(model2, val_loader)
    train_loss.append(train_epoch_loss)
    val_loss.append(val_epoch_loss)
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f"Val Loss: {val_epoch_loss:.4f}")

In [65]:
# Data generation
X_test=1/np.random.gamma(2,2,256)
A = np.random.gamma(2,2,256)
Y_test=A*X_test
Y_test =torch.from_numpy(Y_test).float()
Y_test= Y_test.to(device)
Y_test=Y_test.reshape(256,1)
X_test =torch.from_numpy(X_test).float()
X_test= X_test.to(device)

In [ ]:
mu_theta_test,log_var_theta_test, mu_test, log_var_test =model1(torch.log(Y_test))
#beta_test,alpha_t_test,beta_t_test = model2(Y_test)

In [ ]:
print(beta_test)
print(alpha_t_test)
print(beta_t_test)

In [ ]:
#Score entropie :
SE_gauss = torch.mean(0.5*(mu_test-torch.log(X_test))**2/(torch.exp(log_var_test))+log_var_test+0.5*np.log(2*np.pi))
#SE_invG = -torch.mean(2*torch.log(beta_test)-np.log(math.gamma(2))+3/X_test-beta_test/X_test)
#print(SE_invG)
print(SE_gauss)

# Version 2

Minimisation direct du ELBO à l'aide de l'équation approchée de Philippe. On suppose encore les paramètres de l'a priori connus

In [168]:
features = 2
# define a simple linear VAE
class Linear_invG(nn.Module):
    def __init__(self):
        super(Linear_invG, self).__init__()
 
        # encoder
        self.enc1 = nn.Linear(in_features=1, out_features=5)
        self.enc2 = nn.Linear(in_features=5, out_features=features)
 
    def forward(self, x):
        # encoding
        x = F.relu(self.enc1(x))
        x = self.enc2(x)
        x=x.view(-1,2,1)
        alpha_q = x[:,0,:]**2
        beta_q = x[:,1,:]**2
        return alpha_q, beta_q

In [169]:
model3 = Linear_invG().to(device)
optimizer = optim.Adam(model3.parameters(), lr=0.0001)

def ELBO_loss(data,alpha_q,beta_q):  
    cost =-torch.mean(-3*torch.log(beta_q)+(3-alpha_q)*torch.digamma(alpha_q)+alpha_q*(1-(data+0.5)/beta_q)+torch.lgamma(alpha_q)) 
    return cost

In [170]:
def fit(model, dataloader):
    model.train()
    running_loss = 0.0
    for i, batch in tqdm(enumerate(dataloader), total=int(len(train_data)/dataloader.batch_size)):
        batch=batch[0]
        batch= batch
        batch = batch.to(device)
        batch = batch.view(batch.size(0), -1)
        optimizer.zero_grad()
        alpha_q,beta_q = model(batch)
        loss = ELBO_loss(batch,alpha_q, beta_q)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    train_loss = running_loss/len(dataloader.dataset)
    return train_loss

In [171]:
def validate(model, dataloader):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for i, batch in tqdm(enumerate(dataloader), total=int(len(val_data)/dataloader.batch_size)):            
            batch=batch[0]
            batch = batch.to(device)
            batch = batch.view(batch.size(0), -1)
            alpha_q,beta_q = model(batch)
            loss = ELBO_loss(batch,alpha_q, beta_q)
            running_loss += loss.item()
        
            # save the last batch input and output of every epoch
            #if i == int(len(val_data)/dataloader.batch_size) - 1:
             #   num_rows = 8
              #  both = torch.cat((data.view(batch_size, 1, 28, 28)[:8], 
               #                   reconstruction.view(batch_size, 1, 28, 28)[:8]))
              #  save_image(both.cpu(), f"../outputs/output{epoch}.png", nrow=num_rows)
    val_loss = running_loss/len(dataloader.dataset)
    return val_loss

In [172]:
train_loss = []
val_loss = []
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    print(train_loader)
    train_epoch_loss = fit(model3, train_loader)
    val_epoch_loss = validate(model3, val_loader)
    train_loss.append(train_epoch_loss)
    val_loss.append(val_epoch_loss)
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f"Val Loss: {val_epoch_loss:.4f}")

  0%|          | 0/45 [00:00<?, ?it/s]

Epoch 1 of 1000
Train Loss: 0.1365
Val Loss: 0.1259
Epoch 2 of 1000


  0%|          | 0/45 [00:00<?, ?it/s]

Train Loss: 0.1234
Val Loss: 0.1142
Epoch 3 of 1000
Train Loss: 0.1129
Val Loss: 0.1047
Epoch 4 of 1000


  0%|          | 0/45 [00:00<?, ?it/s]

...

Train Loss: 0.0139
Val Loss: 0.0111
Epoch 997 of 1000
Train Loss: 0.0139
Val Loss: 0.0111
Epoch 998 of 1000


  0%|          | 0/45 [00:00<?, ?it/s]

Train Loss: 0.0139
Val Loss: 0.0111
Epoch 999 of 1000
Train Loss: 0.0139
Val Loss: 0.0111
Epoch 1000 of 1000


100%|██████████| 5/5 [00:00<00:00, 1153.93it/s]

Train Loss: 0.0139
Val Loss: 0.0111


In [176]:
alpha_q,beta_q = model3(Y_test)
SE_ELBO =-torch.mean(alpha_q*torch.log(beta_q)-torch.lgamma(alpha_q)+(alpha_q+1)/X_test-beta_q/X_test)
print(SE_ELBO)

tensor(-3.2566, device='cuda:0', grad_fn=<NegBackward>)


In [177]:
print(alpha_q,beta_q)

tensor([[2.9578],
        [2.9332],
        [2.9616],
        [3.0405],
        [2.9603],
        [2.9549],
        [3.1086],
        [2.9724],
        [2.9721],
        [2.9354],
        [2.9585],
        [2.9632],
        [2.9641],
        [2.9699],
        [2.9628],
        [3.4649],
        [2.9553],
        [2.9698],
        [2.9675],
        [3.0116],
        [3.1962],
        [2.9692],
        [2.9620],
        [2.9506],
        [2.9685],
        [2.9621],
        [2.9625],
        [2.9625],
        [3.2986],
        [2.9695],
        [2.9648],
        [2.9535],
        [2.9323],
        [2.9675],
        [2.9672],
        [2.9412],
        [2.9369],
        [2.9499],
        [2.9400],
        [2.9653],
        [2.9501],
        [2.9668],
        [2.9641],
        [2.9523],
        [2.9396],
        [2.9635],
        [2.9403],
        [2.9492],
        [2.9678],
        [5.0123],
        [2.9633],
        [3.4026],
        [2.9453],
        [2.9477],
        [2.9331],
        [2

# Version 3

On s'intéresse directement à une reparamétrisation par inversion de la cdf, les paramètres de l'a priori restent connus

In [94]:
class gam_inc_inv(torch.autograd.Function):
    @staticmethod
    def forward(ctx,x,y):
        #res = torch.from_numpy(sc.gammaincinv(x.cpu(),y.cpu()))
        res = (sc.gammaincinv(x.cpu(),y.cpu()/sc.gamma(x.cpu()))).to(device)
        ctx.save_for_backward(x,y,res)
        return res

    @staticmethod
    def backward(ctx, grad_output):
        x,y,res = ctx.saved_tensors
        grad_y = 1/(res**(x-1)*torch.exp(-res))
        mei = torch.zeros((x.shape)).to(device)
        for i in range(x.shape[0]):
            mei[i] = float(mpmath.meijerg([[], [0,0]], [[-x[i,0].tolist()-1, -1,-1], []], res[i,0].tolist()))
        u1 = sc.gammainc(x.cpu(),res.cpu()).to(device)
        u2 = sc.gamma(x.cpu()).to(device)
        u3 = sc.digamma(x.cpu()).to(device)
        grad_x = (torch.log(res)+torch.log(u1)+torch.lgamma(x)+res*mei-u2*u3)/grad_y
        return grad_x,grad_y


In [95]:
features = 2
# define a simple linear VAE
class LinearVAE_invG_rep2(nn.Module):
    def __init__(self):
        super(LinearVAE_invG_rep2, self).__init__()
        
        self.beta_p = 0.5
 
        # encoder
        self.enc1 = nn.Linear(in_features=1, out_features=5)
        self.enc2 = nn.Linear(in_features=5, out_features=features)
 
        # decoder 
        self.dec1 = nn.Linear(in_features=1, out_features=5)
        self.dec2 = nn.Linear(in_features=5, out_features=features)
    
    def reparameterize(self, alpha, beta):
        """
        :param alpha: 2
        :param beta:  from the encoder's latent space
        """
        
        u = torch.rand_like(beta)
        sample = beta/gam_inc_inv.apply(alpha,torch.exp(torch.lgamma(alpha))*u) # sampling as if coming from the input space
        if torch.sum(torch.isnan(sample))>0 : 
            print("Sample:")
            print(sample)
            ind = torch.where(torch.isnan(sample))
            ind_0= ind[0]
            print(ind_0)
            print(beta[ind_0,0])
            print(alpha[ind_0,0])
            print(u[ind_0,0])
            print(torch.exp(torch.lgamma(alpha[ind_0,0]))*u[ind_0,0])
            print(gam_inc_inv.apply(alpha[ind_0,0],torch.exp(torch.lgamma(alpha[ind_0,0]))*u[ind_0,0]))
        sample = torch.nan_to_num(sample)
        return sample
 
    def forward(self, x):
        # encoding
        x = F.relu(self.enc1(x))
        x = self.enc2(x).view(-1, 2, 1)
        # get `mu` and `log_var`
        alpha_q = torch.abs(x[:,0,:])+1
        beta_q = x[:,1,:]**2 
        z = self.reparameterize(alpha_q,beta_q)
        # decoding
        x = F.relu(self.dec1(z))
        reconstruction = self.dec2(x).view(-1, 2, 1)
        alpha_theta = torch.abs(reconstruction[:, 0, :])
        beta_theta = torch.abs(reconstruction[:, 1, :]) 
        return alpha_q, beta_q, alpha_theta, beta_theta

In [96]:
model4 = LinearVAE_invG_rep2().to(device)
optimizer = optim.Adam(model4.parameters(), lr=0.001)

def entropy_loss(data, alpha_t,beta_t):  
    cost = torch.mean((alpha_t-1)* torch.log(data)-beta_t*data+alpha_t*torch.log(beta_t) -torch.lgamma(alpha_t))
    return(cost)

In [97]:
def final_loss(ES_loss, alpha_q, beta_q):
    """
    This function will add the reconstruction loss (BCELoss) and the 
    KL-Divergence.
    KL-Divergence = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    :param bce_loss: recontruction loss
    :param mu: the mean from the latent vector
    :param logvar: log variance from the latent vector
    """
    ESL = -ES_loss 
    KLD = torch.mean(alpha_q+torch.log(beta_q)+torch.lgamma(alpha_q)-(1+alpha_q)*torch.digamma(alpha_q)+3*(torch.log(beta_q)-torch.digamma(alpha_q))+alpha_q/2*beta_q)
    return ES_loss + KLD

In [98]:
def fit(model, dataloader):
    model.train()
    running_loss = 0.0
    for i, batch in tqdm(enumerate(dataloader), total=int(len(train_data)/dataloader.batch_size)):
        batch=batch[0]
        batch= batch
        batch = batch.to(device)
        batch = batch.view(batch.size(0), -1)
        optimizer.zero_grad()
        alpha_q, beta_q, alpha_t, beta_t = model(batch)
        ES_loss = entropy_loss(batch,alpha_t,beta_t)
        loss = final_loss(ES_loss, alpha_q, beta_q)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    train_loss = running_loss/len(dataloader.dataset)
    return train_loss

In [99]:
def validate(model, dataloader):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for i, batch in tqdm(enumerate(dataloader), total=int(len(val_data)/dataloader.batch_size)):            
            batch=batch[0]
            batch = batch.to(device)
            batch = batch.view(batch.size(0), -1)
            alpha_q, beta_q, alpha_t, beta_t = model(batch)
            ES_loss = entropy_loss(batch,alpha_t,beta_t)
            loss = final_loss(ES_loss, alpha_q, beta_q)
            running_loss += loss.item()
        
            # save the last batch input and output of every epoch
            #if i == int(len(val_data)/dataloader.batch_size) - 1:
             #   num_rows = 8
              #  both = torch.cat((data.view(batch_size, 1, 28, 28)[:8], 
               #                   reconstruction.view(batch_size, 1, 28, 28)[:8]))
              #  save_image(both.cpu(), f"../outputs/output{epoch}.png", nrow=num_rows)
    val_loss = running_loss/len(dataloader.dataset)
    return val_loss

In [104]:
train_loss = []
val_loss = []
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    print(train_loader)
    train_epoch_loss = fit(model4, train_loader)
    val_epoch_loss = validate(model4, val_loader)
    train_loss.append(train_epoch_loss)
    val_loss.append(val_epoch_loss)
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f"Val Loss: {val_epoch_loss:.4f}")

  2%|▏         | 1/45 [00:00<00:08,  5.14it/s]

Epoch 1 of 1000


  2%|▏         | 1/45 [00:00<00:08,  5.17it/s]

Train Loss: -0.7681
Val Loss: -0.7379
Epoch 2 of 1000


  2%|▏         | 1/45 [00:00<00:08,  5.16it/s]

...

  0%|          | 0/45 [00:00<?, ?it/s]

Train Loss: -558.2545
Val Loss: -419.7118
Epoch 78 of 1000


 89%|████████▉ | 40/45 [00:08<00:01,  4.93it/s]

Sample:
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], device='cuda:0', grad_fn=<DivBa

ValueError: 

In [105]:
alpha_q,beta_q,alpha_test,beta_test = model4(Y_test)
print(alpha_q)
print(beta_q)
print(alpha_test)
print(beta_test)
SE_invG_rep2 = -torch.mean(alpha_q*torch.log(beta_q)-torch.lgamma(alpha_q)-(alpha_q+1)*torch.log(X_test)-beta_q/X_test)
print(SE_invG_rep2)

Sample:
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
  

In [90]:
import mpmath

s=2
x=0.5

#t = torch.tensor([[1], [2],[3],[4]])
t = torch.tensor([1,2,3,4])
#t = mpmath.mpf(t.detach().numpy())

u=mpmath.meijerg([[], [0,0]], [[-s-1, -1,-1], []], t[0].tolist())
x=float(u)
print(type(x))
torch.Tensor([x])

<class 'float'>


tensor([0.0295])

# Tensorflow VAE

### Import packages

In [8]:
!pip install tensorflow

In [12]:
!activate tensorflow
import tensorflow as tf
#import tensorflow_probability as tfp

Could not find conda environment: tensorflow
You can list all discoverable environments with `conda info --envs`.



ModuleNotFoundError: No module named 'tensorflow'

/anaconda/bin/activate: 5: /anaconda/envs/py38_tensorflow/etc/conda/activate.d/activate-binutils_linux-64.sh: Syntax error: "(" unexpected


ModuleNotFoundError: No module named 'tensorflow'